In [1]:
# MiniProyecto 4: Optimización de parámetros

¡Bienvenidos al cuarto (mini)proyecto de la carrera de Data Science de Acamica! 

En este proyecto vamos a seguir trabajando (por última vez) con el dataset de propiedades en venta publicadas en el portal [Properati](www.properati.com.ar). El objetivo en este caso es optimizar los parámetros de los algoritmos que usamos en el proyecto pasado.

El dataset es el mismo del proyecto 3. Recordemos que las columnas que se agregan son:

* `barrios_match`: si coincide el barrio publicado con el geográfico vale 1, si no 0.

* `PH`, `apartment`, `house`: variables binarias que indican el tipo de propiedad.

* dummies de barrios: variables binarias con 1 o 0 según el barrio.

La métrica que vamos a usar para medir es RMSE (raíz del error cuadréatico medio), cuya fórmula es:

$$RMSE = \sqrt{\frac{\sum_{t=1}^n (\hat y_t - y_t)^2}{n}}$$

## Pandas - Levantamos el dataset

In [13]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
path_dataset = '../Data/datos_properati_limpios_model.csv'
df = pd.read_csv(path_dataset)

In [14]:
df.shape

(6376, 59)

In [15]:
df.head()

,lat,lon,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,rooms,barrio_match,PH,apartment,house,...,VILLA LUGANO,VILLA LURO,VILLA ORTUZAR,VILLA PUEYRREDON,VILLA REAL,VILLA RIACHUELO,VILLA SANTA RITA,VILLA SOLDATI,VILLA URQUIZA,outlier_price_m2
0,-34.589,-58.417,170000.000,40.000,38.000,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,-34.591,-58.418,90000.000,27.000,27.000,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,-34.587,-58.437,150000.000,44.000,44.000,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,-34.593,-58.428,154000.000,58.000,58.000,2,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-34.593,-58.428,154000.000,58.000,58.000,3,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


**Separá** el dataset en entrenamiento (80%) y test (20%) utilizando como target la columna `price_aprox_usd`

In [46]:
# Hacé la separación en esta celda
import numpy as np
np.random.seed(123)
from sklearn.model_selection import train_test_split
X = df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print(X_train.shape[0], X_test.shape[0])

5100 1276


## Scikit-learn - Entrenamiento

Para repasar los parámetros de árboles de decisión en Scikit-learn: 

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

En primer lugar veamos como hacer cross validation. Para eso necesitamos definir la cantidad de folds, en este caso vamos a usar 5.

GridSearchCV nos permite testear a través de un espacio de búsqueda de parámetros la mejor combinación posible dado un estimador.

Por ejemplo, en este caso probamos la profundidad máxima y la máxima cantidad de features para hacer los split. Ambos entre 1 y 5.
Recordemos que para hacer la optimización scikit-learn usa la métrica `neg_mean_squared_error` en lugar de `mean_squared_error`.

**Creá** una variable `param_grid` con valores del 1 al 5 para los atributos `max_depth` y `max_features`. 

In [63]:
# Creá en esta celda la variable param_grid

In [64]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

In [65]:
param_grid = {'max_depth': [1,2,3,4,5], 'max_features': [1,2,3,4,5]}

In [66]:
param_grid

{'max_depth': [1, 2, 3, 4, 5], 'max_features': [1, 2, 3, 4, 5]}

**Importá** `GridSearchCV` y `DecisionTreeRegressor`.

**Creá** una variable `grid_search` y asignale un `GridSearchCV` que recorra el `param_grid` que creaste con el algoritmos `DecisionTreeRegressor` y el un scoring de `neg_mean_squared_error`

In [67]:
# Importa y crea un GridSearchCV en esta celda
modelo = DecisionTreeRegressor()
grid_search = GridSearchCV(modelo, param_grid, cv=5,
                           scoring='neg_mean_squared_error', 
                           return_train_score=True)

A continuación, realizá el `fit` del `grid_search` con el conjunto de entrenamiento

In [68]:
# Hace el fit de grid search en esta celda // Fit es entrenar el modelo con los x y los Y

grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': [1, 2, 3, 4, 5], 'max_features': [1, 2, 3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

Revisemos los resultados. Recordemos que no están expresados en RMSE.

In [69]:
grid_search.scorer_

make_scorer(mean_squared_error, greater_is_better=False)

**Mostrá** los `grid_scores` obtenidos durante el `grid_search`

In [70]:
# Mostrá los grid_scores en esta celda
grid_search.best_params_

{'max_depth': 5, 'max_features': 4}

In [75]:
grid_search.best_estimator_

DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=4,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

De esta manera, el valor con mejor resultado (dado el espacio de búsqueda definido) es `max_depth` 3 y `max_features` 3.

**Mostrá** el mejor score y los mejores parámetros encontrados por `grid_search`

In [56]:
# Mostrás los resultados en esta celda

grid_search.cv_results_

{'mean_fit_time': array([0.00357499, 0.00341053, 0.00319448, 0.00258193, 0.00299101,
        0.00299635, 0.00238819, 0.00319805, 0.0029984 , 0.00319767,
        0.00299697, 0.00298281, 0.00299273, 0.00340009, 0.00280156,
        0.00279827, 0.00302634, 0.00266042, 0.00328631, 0.00339198,
        0.00328617, 0.00299239, 0.0032104 , 0.00299196, 0.00339112]),
 'std_fit_time': array([1.01704899e-03, 1.49587562e-03, 3.98537992e-04, 4.92013663e-04,
        1.45728176e-06, 1.39792195e-05, 7.86808222e-04, 7.46082427e-04,
        1.08833838e-05, 4.08367279e-04, 1.12874517e-05, 6.30462784e-04,
        6.31705772e-04, 5.10428955e-04, 3.87276842e-04, 4.01959481e-04,
        6.50280020e-05, 4.06471013e-04, 3.84861040e-04, 4.99681023e-04,
        3.82785536e-04, 6.23825718e-04, 3.91192642e-04, 2.04925727e-05,
        4.88538730e-04]),
 'mean_score_time': array([0.00100436, 0.00098634, 0.00040426, 0.00080299, 0.00079312,
        0.0003983 , 0.00100422, 0.00059214, 0.00100207, 0.00039401,
        0.00

Convertimos a RMSE.

In [72]:
def nmsq2rmse(score):
    return np.round(np.sqrt(-score), 2)

__Encontrar el mejor modelo para el espacio de búsqueda dado__

* `"min_samples_split": [2, 10, 20]`
* `"max_depth": [None, 2, 5, 10, 15]`
* `"min_samples_leaf": [1, 5, 10, 15]`
* `"max_leaf_nodes": [None, 5, 10, 20]`

In [80]:
param_grid2 = {"min_samples_split": [2, 10, 20], "max_depth": [None, 2, 5, 10, 15],"min_samples_leaf": [1, 5, 10, 15], "max_leaf_nodes": [None, 5, 10, 20] }

In [81]:
param_grid2

{'min_samples_split': [2, 10, 20],
 'max_depth': [None, 2, 5, 10, 15],
 'min_samples_leaf': [1, 5, 10, 15],
 'max_leaf_nodes': [None, 5, 10, 20]}

In [83]:
modelo2 = DecisionTreeRegressor()
grid_search2 = GridSearchCV(modelo, param_grid2, cv=5,
                           scoring='neg_mean_squared_error', 
                           return_train_score=True)

In [84]:

grid_search2.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'min_samples_split': [2, 10, 20], 'max_depth': [None, 2, 5, 10, 15], 'min_samples_leaf': [1, 5, 10, 15], 'max_leaf_nodes': [None, 5, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [85]:
grid_search2.best_params_

{'max_depth': 10,
 'max_leaf_nodes': None,
 'min_samples_leaf': 15,
 'min_samples_split': 2}

In [86]:
grid_search2.best_estimator_

DecisionTreeRegressor(criterion='mse', max_depth=10, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=15,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [90]:
grid_search2.best_score_

-459407773.74392384

In [91]:
def nmsq2rmse(score):
    return np.round(np.sqrt(-score), 2)

In [93]:
optimised_decision_tree2 = grid_search2.best_estimator_

In [108]:
from sklearn.metrics import mean_squared_error
y_opt_pred2 = optimised_decision_tree2.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_opt_pred2))
np.round(rmse)

21301.0

In [109]:
val_real2 = pd.Series(y_test.values)  #Series: 
val_pred2 = pd.Series(y_opt_pred2)

In [110]:
predicciones2 = pd.concat([val_real2.rename('Valor real2'),val_pred2.rename('Valor Pred2') ,abs(val_real2-val_pred2).rename('Dif(+/-)')] ,  axis=1)

In [111]:
predicciones2.head(10)

,Valor real2,Valor Pred2,Dif(+/-)
0,80000.000,103438.660,23438.660
1,128000.000,135705.882,7705.882
2,150000.000,156075.759,6075.759
3,85000.000,102400.991,17400.991
4,135000.000,135571.622,571.622
5,135000.000,109560.000,25440.000
6,68000.000,75181.250,7181.250
7,110000.000,140444.444,30444.444
8,134000.000,158431.250,24431.250
9,110000.000,76701.202,33298.798


Recordemos que `GridSearchCV` tiene como parámetro default `refit=True`. Esto significa que luego de hacer la corrida se ajusta el mejor modelo al conjunto de datos de entrada. De esta manera, se puede predecir directamente usando `best_estimator_`.

In [112]:
optimised_decision_tree = grid_search.best_estimator_

__Evaluemos en testing el desempeño de este modelo.__

Como venimos trabajando, el resultado en testing será la medición que usaremos como benchmark para comparar este modelos con otros en el futuro, puesto que no estuvo en contacto con el dataset de test para la calibración.

In [117]:
from sklearn.metrics import mean_squared_error
y_opt_pred = optimised_decision_tree.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_opt_pred))
np.round(rmse)

29900.0

Vemos los primeros 10 resultados de la predicción del valor de propiedades.

In [114]:
val_real = pd.Series(y_test.values)  #Series: 
val_pred = pd.Series(y_opt_pred)

In [115]:
predicciones = pd.concat([val_real.rename('Valor real'),val_pred.rename('Valor Pred') ,abs(val_real-val_pred).rename('Dif(+/-)')] ,  axis=1)

In [116]:
predicciones.head(10)

,Valor real,Valor Pred,Dif(+/-)
0,80000.000,102739.402,22739.402
1,128000.000,128915.706,915.706
2,150000.000,128915.706,21084.294
3,85000.000,102739.402,17739.402
4,135000.000,128915.706,6084.294
5,135000.000,132781.250,2218.750
6,68000.000,114427.136,46427.136
7,110000.000,135185.886,25185.886
8,134000.000,114427.136,19572.864
9,110000.000,102739.402,7260.598
